Probability of subsequence in a random sequence
--------------------------------------------

A random sequence of length $N$ is a sequence, where in each place bases `A`, `T`, `C`, `G` can occur with equal probability.

The probability that a certain subsequence is found in a random sequence of length $N$ is the number random sequences with the subsequence divided by the total number or possible random sequences of such length.

The total number of possible random sequences is $4^N$

How to calculate $S$, the number of random sequences of length $N$ that contain our subsequence?

Our subsequence is set, it does not allow for any change in itself. But if it occurs somewhere in the sequence, then all other bases could be whatever they like—all count toward $S$.

So, we can think of our subsequence as one unit of length $L$. If it occurs once in the sequence, then there is $N-L$ bases left. We can place our unit of subsequence anywhere between those $N-L$ bases, there are $\binom{N-L+1}{1}$ ways to do so. The total number of random sequences with our subsequence is $4^{N-L} \cdot \binom{N-L+1}{1}$ 

However we counted some sequences multiple times: for instance, if our subsequence started at base 0, and among all the random sequences of the rest $N-L$ bases there was a sequence with our subsequence also starting at $L$. And we counted that sequence twice, because we also counted it when considering that our subsequence was at position $L$ and the random beginning turned our to be our sequence again. So we need to subtract the number of random sequences with two subsequences from the previous result.

The number of possible sequences which contain two subsequences is $4^{N-2L} \cdot \binom{N-2L+2}{2}$ and we subtract it from the original estimate to get 

$$4^{N-L} \cdot \binom{N-L+1}{1} - 4^{N-2L} \cdot \binom{N-2L+2}{2}$$

However now we subtracted too much: the random sequence which has three our subsequences was accounted for 3 times in the left part and 3 times in the right part. So we need to add back the number of random sequences with 3 our subsequences. Extrapolating this, we get the total number of random sequences computed like this:

$$\sum_{k=1}^{N/L} -1^{k+1} \cdot 4^{N-kL} \cdot \binom{N-kL+k}{k}$$

### Code

Some basic functions to generate random DNA, check if a sequence is in a DNA and estimate probability based on random generation

In [20]:
import random
import scipy.misc
import numpy as np

DNA_BASES = ('A', 'T', 'C', 'G')

def random_dna(l):
    return [random.choice(DNA_BASES) for i in range(l)]

def has_sequence(dna, seq):
    if len(seq) > len(dna):
        return False
    i = None
    while True:
        try:
            if i is None:
                start = 0
            else:
                start = i + 1
            i = dna.index(seq[0], start)
        except ValueError:
            return False
        if len(dna) - i < len(seq):
            return False
        all_ok = True
        for j in range(1, len(seq)):
            if dna[i + j] != seq[j]:
                all_ok = False
                break
        if all_ok:
            return True

def estimate_probability_of_seq(random_length, needle, number_of_trials):
    c = 0
    for q in range(number_of_trials):
        if has_sequence(random_dna(random_length), needle):
            c += 1
    return c / number_of_trials

A function to compute one part of the sum above

In [8]:
def number_of_sequences(random_length, needle_length, needle_count):
    return scipy.misc.comb(random_length - needle_count * needle_length + needle_count, needle_count) \
            * 4 ** (random_length - needle_length * needle_count) \
            * (-1) ** (needle_count + 1)

A function to compute the whole probability

In [25]:
def probability_of_seq(random_length, needle_length):
    number_of_seqs = np.sum(
        [number_of_sequences(random_length, needle_length, i) 
         for i in range(1, 1 + random_length // needle_length)]
    )
    total_number = 4**random_length
    return number_of_seqs / total_number

### Results

Our predicted probability works fine

In [18]:
probability_of_seq(100, 3)

0.7968880041718008

In [23]:
estimate_probability_of_seq(100, ['C', 'T', 'G'], 20000)

0.7983

But we have a problem

In [24]:
emulate_probability_of_seq(100, ['C', 'A', 'C'], 20000)

0.77765

This is because our needle sequence can overlap itself. This means that when we count the number of random sequences of length 5, for instance, we count sequence `CACAC` twice, but when we subtract the number of sequences which has two `CAC` subsequences, we subtract 0. How to count them properly?

We can count overlapping sequences as a single sequence. Let's count the number of random sequences of length 6 that contain two `CAC` with overlaps. If there is no overlap, we do it as previously descibed: $4^{6-2\cdot3} \cdot \binom{6 - 2\cdot3+2}{2} = 1$. If there is an overlap, we know the length of the overlap, in this case it is 1, so the length of our 2 needles together is $2 \cdot 3 - 1 = 5$. We count that as a single sequence and the number of possible random sequences is computed in the same way as before: $4^{6-5} \cdot \binom{6-5+1}{1} = 4 \cdot 2 = 8$. And we add them together and get 9 possible sequences.

Let's test it

In [34]:
emulate_probability_of_seq(6, ['C', 'A', 'C'], 20000)

0.05975

In [35]:
number_of_sequences(6, 3, 1) / 4**6

0.0625

In [36]:
(number_of_sequences(6, 3, 1) - 9) / 4**6

0.060302734375

Let $C_2$ be the number of random sequences that contain our sequence twice _with_ overlaps. Let $O_1, O_2 … O_i$ be the lengths of all possible $i$ overlaps for our sequence. Then,

$$C_2 = 4^{N-2L} \cdot \binom{N-2L+2}{2} + 4^{N-2L+O_1} \cdot \binom{N-2L+O_2+1}{1} + … 
  + 4^{N-2L+O_i} \cdot \binom{N-2L+O_i+1}{1}$$

We can rewrite this as a sum:

$$C_2 = 4^{N-2L} \cdot \binom{N-2L+2}{2} + \sum_{j=0}^i 4^{N-2L+O_j+1}{1}$$

For $C_3$ we need to consider the number of ways to form subsequences with overlaps: we can have a single sequence with 2 overlaps, or 1 sequence with 1 overlap and 1 needle or we can have 3 needles.